# Combining Datasets: Concat and Append
_____________________

``Series`` and ``DataFrame``s are built for operations on data which

* come from different data sources

* involve anything **from** very straightforward **concatenation** of two different datasets, **to** more complicated **joins** and **merges** that correctly handle any overlaps between the datasets

* are fast and straightforward with using Pandas functions and methods  

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

In [ ]:
# example DataFrame
make_df('ABC', range(3))

Class for displaying multiple ``DataFrame``s side by side with ``_repr_html_`` method, which IPython uses to display:

In [ ]:
class display:
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

## 1. Simple Concatenation with ``pd.concat()``
------------------------

#### 1.1. Concatenation of ``Series`` objects
-----------------

* The combination of options of the ``pd.concat()`` allows a wide range of possible behaviors when **joining** two datasets

In [ ]:
pd.concat?

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

In [ ]:
ser2.append(ser1)

In [ ]:
pd.concat([ser2, ser1])

#### 1.2. Concatenation of ``DataFrame`` objects
-------------

* By default, the concatenation takes place row-wise within the ``DataFrame`` (i.e., ``axis=0``)

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

* Specification of an axis along which concatenation will take place:

In [ ]:
df3 = make_df('AB', [1, 2])
df4 = make_df('CD', [1, 2])
display('df3', 'df4', "pd.concat([df3, df4], axis=1)")

In [ ]:
display('df3', 'df4', "pd.concat([df3, df4])")

In [ ]:
df5 = make_df('AB', [2, 3])
display('df1', 'df5', 'pd.concat([df1, df5], axis=1)')

## 2. Duplicate indices and ways to handle it
_____________________________________________

#### 2.1. Preserving indices
-----------

* Pandas concatenation *preserves indices*, even if the result will have duplicate indices (important difference between ``np.concatenate`` and ``pd.concat`` ): 

In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

* ``verify_integrity=True`` -- catching the repeats as an error:  the concatenation will raise an exception if there are duplicate indices:

In [ ]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

#### 2.2. Managing of duplication
-----------------
* ``ignore_index=True`` -- ignoring the index when itself does not matter: the concatenation will create a new integer index for the resulting ``Series``:

In [ ]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

* Adding MultiIndex ``keys`` option to specify a label for the data sources; the result is a multiply indexed DataFrame:

In [ ]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

In [ ]:
dxy = pd.concat([x, y], keys=['x', 'y'])
dxy

In [ ]:
dxy.loc['x']

#### 2.3. Concatenation in case of different sets of column names with joins 
--------------

* ``join='outer', join_axes=None`` -- default values -- outer joins --  the join is a union of the input columns, the entries for which no data is available are filled with NaN

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6], sort=False)' )

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

* ``join='inner'`` -- inner join -- using the common columns only  

In [ ]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

## 3. Concatenation with ``Series.append()`` and ``DataFrame.append()``  
___________________

* ``Series`` and ``DataFrame`` objects have an ``append()`` **method** for direct concatenation 

* ``append()`` does not modify the original object – instead it creates **a new** object  (unlike ``append()`` and ``extend()``  of `` lists``)

* ``append()`` also **is not a very efficient**, because it involves creation of a new index *and* data buffer.

* In the case of multiple ``append`` operations, it is generally better to build a list of ``DataFrame``s and pass them all at once to the ``concat()`` function

In [ ]:
pd.Series.append?

In [ ]:
pd.DataFrame.append?

In [ ]:
display('df1', 'df2', 'df1.append(df2)')

In [ ]:
display('x', 'y', 'x.append(y, ignore_index=True)')

In [ ]:
display('x')